# How does **GROUPBY** happen in **DASK** (distributed framework)?

<!-- ![title](img/panda.png) -->
<img src="img/panda.png" width=800 />

In [3]:
import pandas as pd
import dask.dataframe as dd

In [4]:
pdf = pd.DataFrame(dict(a=[1, 1, 2, 3, 3, 1, 1, 2, 3, 3, 4, 4, 1, 3, 6],b=[1, 3, 10, 3, 2, 1, 3, 10, 3, 3, 12, 0, 9, 2, 4],c=[2, 4, 5, 2, 3, 5, 2, 3, 9, 2, 44, 33, 2, 4,1]))
pdf

,a,b,c
0,1,1,2
1,1,3,4
2,2,10,5
3,3,3,2
4,3,2,3
5,1,1,5
6,1,3,2
7,2,10,3
8,3,3,9
9,3,3,2


In [5]:
ddf = dd.from_pandas(pdf, npartitions=3)
ddf

,a,b,c
npartitions=3,,,
0,int64,int64,int64
5,...,...,...
10,...,...,...
14,...,...,...


In [6]:
ddf.partitions[2].compute()

,a,b,c
10,4,12,44
11,4,0,33
12,1,9,2
13,3,2,4
14,6,4,1


In [7]:
sdf1 = pdf[:5]
sdf2 = pdf[5:10]
sdf3 = pdf[10:]

In [8]:
gp1 = sdf1.groupby(['a', 'b']).c.sum()
gp2 = sdf2.groupby(['a', 'b']).c.sum()
gp3 = sdf3.groupby(['a', 'b']).c.sum()

In [9]:
gp1,gp2,gp3

(a  b 
 1  1     2
    3     4
 2  10    5
 3  2     3
    3     2
 Name: c, dtype: int64,
 a  b 
 1  1      5
    3      2
 2  10     3
 3  3     11
 Name: c, dtype: int64,
 a  b 
 1  9      2
 3  2      4
 4  0     33
    12    44
 6  4      1
 Name: c, dtype: int64)

In [10]:
concated_gp = pd.concat([gp1, gp2, gp3])

In [11]:
concated_gp

a  b 
1  1      2
   3      4
2  10     5
3  2      3
   3      2
1  1      5
   3      2
2  10     3
3  3     11
1  9      2
3  2      4
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

In [12]:
total = concated_gp.groupby(level=[0, 1]).sum()

In [13]:
total

a  b 
1  1      7
   3      6
   9      2
2  10     8
3  2      7
   3     13
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

In [14]:
pdf.groupby(['a', 'b']).c.sum()

a  b 
1  1      7
   3      6
   9      2
2  10     8
3  2      7
   3     13
4  0     33
   12    44
6  4      1
Name: c, dtype: int64

In [15]:
ddf.groupby(['a', 'b']).c.sum().compute()

a  b 
1  1      7
   3      6
2  10     8
3  2      7
   3     13
1  9      2
4  0     33
   12    44
6  4      1
Name: c, dtype: int64